In [ ]:
import cv2
import time
import pyttsx3
import threading
import queue  # Queue to handle speech requests sequentially
import HandTrackingModule as htm

# Initialize pyttsx3 engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)

# Queue for speech requests
speech_queue = queue.Queue()

# Mapping numbers to words
number_words = ["Zero", "One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten"]

# Webcam setup
wCam, hCam = 640, 480
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Webcam not detected.")
    exit()
cap.set(3, wCam)
cap.set(4, hCam)

pTime = 0
detector = htm.handDetector(detectionCon=0.75)
prevFingers = -1


def speech_worker():
    while True:
        text = speech_queue.get() 
        if text is None:
            break 
        engine.say(text)
        engine.runAndWait()
        speech_queue.task_done()  


speech_thread = threading.Thread(target=speech_worker, daemon=True)
speech_thread.start()

while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)
    img = detector.findHands(img)
    lmList, bbox = detector.findPosition(img, draw=False)

    totalFingers = 0
    if lmList:
        fingersUp = detector.fingersUp()
        totalFingers = fingersUp.count(1)

       
        if bbox:
            cv2.putText(img, "Detected Hand", (bbox[0], bbox[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        
        cv2.putText(img, number_words[totalFingers], (45, 375), 
                    cv2.FONT_HERSHEY_PLAIN, 5, (255, 0, 0), 5)

       
        if totalFingers != prevFingers:
            prevFingers = totalFingers
            speech_queue.put(number_words[totalFingers]) 

   
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
   

   
    cv2.imshow("Webcam Feed", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):  
        break


speech_queue.put(None) 
speech_thread.join()

cap.release()
cv2.destroyAllWindows()
